In [1]:
import nltk
import string
import re
import numpy as np

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

# Define the list of stop words
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\momos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\momos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Splitting and unique words

In [2]:
with open('HP1.txt', 'r', encoding='utf-8') as file:
    # Read the entire text
    text = file.read()

# Step 1: Tokenize the text and remove punctuation
wordTokens = word_tokenize(text)
words_no_punct = [word for word in wordTokens if word not in string.punctuation]

# Comment out this section if you do not want to remove stop words
words_no_stopwords = [word for word in words_no_punct if word.lower() not in stop_words]

# Collect unique words
uniqueWords = []
for word in words_no_stopwords:
    if word not in uniqueWords:
        uniqueWords.append(word)

print(uniqueWords)

['Mr.', 'Mrs.', 'Dursley', 'number', 'four', 'Privet', 'Drive', 'proud', 'say', 'perfectly', 'normal', 'thank', 'much', 'last', 'people', '’', 'expect', 'involved', 'anything', 'strange', 'mysterious', 'hold', 'nonsense', 'director', 'firm', 'called', 'Grunnings', 'made', 'drills', 'big', 'beefy', 'man', 'hardly', 'neck', 'although', 'large', 'mustache', 'thin', 'blonde', 'nearly', 'twice', 'usual', 'amount', 'came', 'useful', 'spent', 'time', 'craning', 'garden', 'fences', 'spying', 'neighbors', 'small', 'son', 'Dudley', 'opinion', 'finer', 'boy', 'anywhere', 'Dursleys', 'everything', 'wanted', 'also', 'secret', 'greatest', 'fear', 'somebody', 'would', 'discover', 'think', 'could', 'bear', 'anyone', 'found', 'Potters', 'Potter', 'sister', 'met', 'several', 'years', 'fact', 'pretended', 'good-for-nothing', 'husband', 'unDursleyish', 'possible', 'shuddered', 'arrived', 'street', 'knew', 'never', 'even', 'seen', 'another', 'good', 'reason', 'keeping', 'away', 'want', 'mixing', 'child', '

One hot encoding

In [3]:
#create a dictionary with a 1 hot representation of the words
wordDict = {}
for word in uniqueWords:
    wordDict[word] = np.zeros(len(uniqueWords), dtype=int) 
    wordDict[word][uniqueWords.index(word)] = 1

Word context pairs

In [4]:
wordContextPairs = []
contextSize = 2

for i in range(len(wordTokens)):
    mainWord = wordTokens[i]
    contextIndices = list(range(max(0, i - contextSize), i)) + list(range(i + 1, min(len(wordTokens), i + contextSize + 1)))
    for indices  in contextIndices:
        contextWord = wordTokens[indices]
        wordContextPairs.append((mainWord, contextWord))

NN Stuff

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.initializers import RandomNormal

# Prepare the training data
X = []
y = []

for pair in wordContextPairs:
    if pair[1] in wordDict and pair[0] in wordDict:
        X.append(wordDict[pair[0]])  # target word
        y.append(wordDict[pair[1]])  # context word

X = np.array(X)
y = np.array(y)

# Define the neural network model
vocab_size = len(uniqueWords)
embedding_dim = 50

model = Sequential()
model.add(Dense(embedding_dim, input_dim=vocab_size, activation='linear',
                kernel_initializer=RandomNormal(mean=0.0, stddev=0.05))) 
model.add(Dense(vocab_size, activation='softmax', 
                kernel_initializer=RandomNormal(mean=0.0, stddev=0.05)))  

# Compile the model with a large learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X, y, epochs=200, batch_size=64)

# Outpuyt the weights of the hidden layer
weights = model.layers[0].get_weights()[0]
print(weights)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print(f'Loss: {loss}, Accuracy: {accuracy}')

c:\Users\momos\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.0479 - loss: 15.6595
Epoch 2/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.0498 - loss: 27.8455
Epoch 3/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.0560 - loss: 26.5781
Epoch 4/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.0572 - loss: 25.1129
Epoch 5/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.0606 - loss: 25.5501
Epoch 6/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.0608 - loss: 26.0576
Epoch 7/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.0615 - loss: 26.1542
Epoch 8/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.0601 - loss: 26.2193
Epoch 9/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.0636 - loss: 26.4356
Epoch 10/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.0613 - loss: 26.3744
Epoch 11/200
1386/1386 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.0596 - loss: 26.7020
Epoch 12

In [6]:
# Implement an inference function which receives a 1-hot vector and provides the corresponding embed- ding.
def getEmbedding(OHVector):
    return np.dot(OHVector, weights)

# Take in OHvector from the user
OHVector = input("Enter a one-hot vector: ")
OHVector = np.array([int(x) for x in OHVector.split()])
embedding = getEmbedding(OHVector)
print(embedding)


ValueError: shapes (0,) and (6823,50) not aligned: 0 (dim 0) != 6823 (dim 0)